In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.tag import pos_tag
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import subprocess
import re
import statistics

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# load the new content features data from csv.
df_cv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/content_features_cv1.csv')
df_cv.drop(columns=[df_cv.columns[0]],inplace=True)
df_cv.head()
x = df_cv.drop(columns=["label"]).values
y = df_cv["label"].values

In [11]:
# split the data
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)
# 40 train1, 40 train2, 10 val, 10 test
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=0.5, random_state=42)
# Further split the remaining data (20%) into validation and testing sets (50% each)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(len(X_train1))
print(len(X_train2))
print(len(X_val))
print(len(X_test))


28853
28854
7213
7214


In [12]:
input=X_train1.shape[1]
print(input)

18


This is the best config

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.optimizers import Adamax
from keras.callbacks import ModelCheckpoint

ffnn_neurons = 512
dropout = 0.05

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))


checkpoint_callback = ModelCheckpoint(
    filepath="/content/drive/MyDrive/School/Term 8/AI Project/content_ffnn_model_best_new.h5",
    monitor='val_loss', # or 'val_accuracy' depending on your task
    verbose=1,
    save_best_only=True,
    mode='min' # Use 'min' for loss, 'max' for accuracy
)

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=50, batch_size=32, validation_data=(X_val, y_val),callbacks=[checkpoint_callback])

Epoch 1/50
896/902 [============================>.] - ETA: 0s - loss: 0.4789 - accuracy: 0.7767
Epoch 1: val_loss improved from inf to 0.43847, saving model to /content/drive/MyDrive/School/Term 8/AI Project/content_ffnn_model_best_new.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


902/902 [==============================] - 10s 7ms/step - loss: 0.4787 - accuracy: 0.7768 - val_loss: 0.4385 - val_accuracy: 0.8005
Epoch 2/50
899/902 [============================>.] - ETA: 0s - loss: 0.4432 - accuracy: 0.7959
Epoch 2: val_loss improved from 0.43847 to 0.43012, saving model to /content/drive/MyDrive/School/Term 8/AI Project/content_ffnn_model_best_new.h5
902/902 [==============================] - 4s 5ms/step - loss: 0.4431 - accuracy: 0.7959 - val_loss: 0.4301 - val_accuracy: 0.8052
Epoch 3/50
892/902 [============================>.] - ETA: 0s - loss: 0.4322 - accuracy: 0.7999
Epoch 3: val_loss improved from 0.43012 to 0.41917, saving model to /content/drive/MyDrive/School/Term 8/AI Project/content_ffnn_model_best_new.h5
902/902 [==============================] - 4s 5ms/step - loss: 0.4323 - accuracy: 0.7998 - val_loss: 0.4192 - val_accuracy: 0.8096
Epoch 4/50
898/902 [============================>.] - ETA: 0s - loss: 0.4236 - accuracy: 0.8055
Epoch 4: val_loss improv

In [13]:
# Load the model
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/School/Term 8/AI Project/content_ffnn_model_best_new.h5")
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print("Model accuracy: {:5.2f}%".format(100 * acc))


226/226 - 1s - loss: 0.4091 - accuracy: 0.8143 - 818ms/epoch - 4ms/step
Model accuracy: 81.43%


below are the testing from model tuning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='sigmoid', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='sigmoid'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='sigmoid'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
902/902 [==============================] - 9s 7ms/step - loss: 0.6427 - accuracy: 0.6194 - val_loss: 0.5105 - val_accuracy: 0.7632
Epoch 2/10
902/902 [==============================] - 4s 4ms/step - loss: 0.5427 - accuracy: 0.7332 - val_loss: 0.4974 - val_accuracy: 0.7743
Epoch 3/10
902/902 [==============================] - 4s 5ms/step - loss: 0.5252 - accuracy: 0.7451 - val_loss: 0.4860 - val_accuracy: 0.7769
Epoch 4/10
902/902 [==============================] - 5s 6ms/step - loss: 0.5168 - accuracy: 0.7487 - val_loss: 0.4823 - val_accuracy: 0.7769
Epoch 5/10
902/902 [==============================] - 4s 5ms/step - loss: 0.5089 - accuracy: 0.7565 - val_loss: 0.4754 - val_accuracy: 0.7811
Epoch 6/10
902/902 [==============================] - 4s 4ms/step - loss: 0.5040 - accuracy: 0.7578 - val_loss: 0.4761 - val_accuracy: 0.7800
Epoch 7/10
902/902 [==============================] - 6s 7ms/step - loss: 0.4992 - accuracy: 0.7615 - val_loss: 0.4716 - val_accuracy: 0.7823
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
902/902 [==============================] - 6s 5ms/step - loss: 0.5571 - accuracy: 0.7219 - val_loss: 0.4641 - val_accuracy: 0.7832
Epoch 2/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4937 - accuracy: 0.7696 - val_loss: 0.4545 - val_accuracy: 0.7944
Epoch 3/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4764 - accuracy: 0.7801 - val_loss: 0.4454 - val_accuracy: 0.7969
Epoch 4/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4696 - accuracy: 0.7841 - val_loss: 0.4393 - val_accuracy: 0.8008
Epoch 5/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4614 - accuracy: 0.7908 - val_loss: 0.4344 - val_accuracy: 0.8024
Epoch 6/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4600 - accuracy: 0.7901 - val_loss: 0.4325 - val_accuracy: 0.8049
Epoch 7/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4541 - accuracy: 0.7914 - val_loss: 0.4293 - val_accuracy: 0.8058
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='Softmax', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='Softmax'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='Softmax'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 7s 6ms/step - loss: 0.6928 - accuracy: 0.5125 - val_loss: 0.6931 - val_accuracy: 0.5084
Epoch 2/10
902/902 [==============================] - 4s 5ms/step - loss: 0.6927 - accuracy: 0.5160 - val_loss: 0.6930 - val_accuracy: 0.5084
Epoch 3/10
902/902 [==============================] - 4s 4ms/step - loss: 0.6846 - accuracy: 0.5622 - val_loss: 0.6508 - val_accuracy: 0.7602
Epoch 4/10
902/902 [==============================] - 5s 6ms/step - loss: 0.6212 - accuracy: 0.7096 - val_loss: 0.5483 - val_accuracy: 0.7732
Epoch 5/10
902/902 [==============================] - 4s 5ms/step - loss: 0.5709 - accuracy: 0.7360 - val_loss: 0.5118 - val_accuracy: 0.7800
Epoch 6/10
902/902 [==============================] - 4s 4ms/step - loss: 0.5496 - accuracy: 0.7488 - val_loss: 0.4965 - val_accuracy: 0.7857
Epoch 7/10
902/902 [==============================] - 5s 6ms/step - loss: 0.5339 - accuracy: 0.7587 - val_loss: 0.4849 - val_accuracy: 0.7897
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adagrad', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 5s 4ms/step - loss: 0.7148 - accuracy: 0.5141 - val_loss: 0.6760 - val_accuracy: 0.6343
Epoch 2/10
902/902 [==============================] - 4s 5ms/step - loss: 0.6985 - accuracy: 0.5478 - val_loss: 0.6625 - val_accuracy: 0.7000
Epoch 3/10
902/902 [==============================] - 5s 5ms/step - loss: 0.6863 - accuracy: 0.5711 - val_loss: 0.6509 - val_accuracy: 0.7077
Epoch 4/10
902/902 [==============================] - 4s 4ms/step - loss: 0.6750 - accuracy: 0.5920 - val_loss: 0.6401 - val_accuracy: 0.7176
Epoch 5/10
902/902 [==============================] - 4s 4ms/step - loss: 0.6643 - accuracy: 0.6111 - val_loss: 0.6286 - val_accuracy: 0.7215
Epoch 6/10
902/902 [==============================] - 5s 5ms/step - loss: 0.6587 - accuracy: 0.6257 - val_loss: 0.6176 - val_accuracy: 0.7254
Epoch 7/10
902/902 [==============================] - 4s 5ms/step - loss: 0.6512 - accuracy: 0.6319 - val_loss: 0.6079 - val_accuracy: 0.7280
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 128
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 6s 6ms/step - loss: 0.6400 - accuracy: 0.6383 - val_loss: 0.5219 - val_accuracy: 0.7567
Epoch 2/10
902/902 [==============================] - 4s 5ms/step - loss: 0.5616 - accuracy: 0.7263 - val_loss: 0.4864 - val_accuracy: 0.7747
Epoch 3/10
902/902 [==============================] - 4s 5ms/step - loss: 0.5353 - accuracy: 0.7451 - val_loss: 0.4741 - val_accuracy: 0.7803
Epoch 4/10
902/902 [==============================] - 5s 5ms/step - loss: 0.5150 - accuracy: 0.7584 - val_loss: 0.4661 - val_accuracy: 0.7868
Epoch 5/10
902/902 [==============================] - 5s 5ms/step - loss: 0.5080 - accuracy: 0.7639 - val_loss: 0.4632 - val_accuracy: 0.7930
Epoch 6/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4991 - accuracy: 0.7652 - val_loss: 0.4576 - val_accuracy: 0.7947
Epoch 7/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4898 - accuracy: 0.7725 - val_loss: 0.4523 - val_accuracy: 0.7970
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 256
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 6s 5ms/step - loss: 0.5344 - accuracy: 0.7386 - val_loss: 0.4551 - val_accuracy: 0.7901
Epoch 2/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4780 - accuracy: 0.7745 - val_loss: 0.4463 - val_accuracy: 0.7959
Epoch 3/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4647 - accuracy: 0.7841 - val_loss: 0.4360 - val_accuracy: 0.8013
Epoch 4/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4573 - accuracy: 0.7901 - val_loss: 0.4315 - val_accuracy: 0.8040
Epoch 5/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4536 - accuracy: 0.7927 - val_loss: 0.4286 - val_accuracy: 0.8054
Epoch 6/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4507 - accuracy: 0.7954 - val_loss: 0.4210 - val_accuracy: 0.8113
Epoch 7/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4439 - accuracy: 0.7952 - val_loss: 0.4251 - val_accuracy: 0.8040
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 256
dropout = 0.1

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 7s 6ms/step - loss: 0.4825 - accuracy: 0.7735 - val_loss: 0.4393 - val_accuracy: 0.7991
Epoch 2/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4477 - accuracy: 0.7917 - val_loss: 0.4241 - val_accuracy: 0.8070
Epoch 3/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4368 - accuracy: 0.7988 - val_loss: 0.4215 - val_accuracy: 0.8095
Epoch 4/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4292 - accuracy: 0.8029 - val_loss: 0.4142 - val_accuracy: 0.8124
Epoch 5/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4222 - accuracy: 0.8075 - val_loss: 0.4145 - val_accuracy: 0.8105
Epoch 6/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4178 - accuracy: 0.8070 - val_loss: 0.4119 - val_accuracy: 0.8146
Epoch 7/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4151 - accuracy: 0.8088 - val_loss: 0.4068 - val_accuracy: 0.8169
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 256
dropout = 0.05

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 5s 4ms/step - loss: 0.4804 - accuracy: 0.7725 - val_loss: 0.4462 - val_accuracy: 0.7974
Epoch 2/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4466 - accuracy: 0.7946 - val_loss: 0.4340 - val_accuracy: 0.8022
Epoch 3/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4317 - accuracy: 0.7988 - val_loss: 0.4188 - val_accuracy: 0.8092
Epoch 4/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4258 - accuracy: 0.8035 - val_loss: 0.4170 - val_accuracy: 0.8124
Epoch 5/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4207 - accuracy: 0.8062 - val_loss: 0.4098 - val_accuracy: 0.8164
Epoch 6/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4153 - accuracy: 0.8081 - val_loss: 0.4091 - val_accuracy: 0.8153
Epoch 7/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4096 - accuracy: 0.8122 - val_loss: 0.4081 - val_accuracy: 0.8191
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 256
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 7s 6ms/step - loss: 0.5529 - accuracy: 0.7257 - val_loss: 0.4777 - val_accuracy: 0.7905
Epoch 2/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4931 - accuracy: 0.7719 - val_loss: 0.4468 - val_accuracy: 0.7972
Epoch 3/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4792 - accuracy: 0.7828 - val_loss: 0.4369 - val_accuracy: 0.8006
Epoch 4/10
902/902 [==============================] - 6s 6ms/step - loss: 0.4719 - accuracy: 0.7868 - val_loss: 0.4465 - val_accuracy: 0.8072
Epoch 5/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4657 - accuracy: 0.7916 - val_loss: 0.4553 - val_accuracy: 0.8008
Epoch 6/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4634 - accuracy: 0.7887 - val_loss: 0.4249 - val_accuracy: 0.8084
Epoch 7/10
902/902 [==============================] - 6s 6ms/step - loss: 0.4568 - accuracy: 0.7956 - val_loss: 0.4274 - val_accuracy: 0.8090
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 7s 6ms/step - loss: 0.5088 - accuracy: 0.7574 - val_loss: 0.4605 - val_accuracy: 0.7937
Epoch 2/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4705 - accuracy: 0.7821 - val_loss: 0.4369 - val_accuracy: 0.7984
Epoch 3/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4597 - accuracy: 0.7886 - val_loss: 0.4298 - val_accuracy: 0.8038
Epoch 4/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4535 - accuracy: 0.7907 - val_loss: 0.4287 - val_accuracy: 0.8073
Epoch 5/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4508 - accuracy: 0.7941 - val_loss: 0.4228 - val_accuracy: 0.8108
Epoch 6/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4481 - accuracy: 0.7953 - val_loss: 0.4216 - val_accuracy: 0.8096
Epoch 7/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4436 - accuracy: 0.7961 - val_loss: 0.4178 - val_accuracy: 0.8153
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.05

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 6s 5ms/step - loss: 0.4772 - accuracy: 0.7760 - val_loss: 0.4478 - val_accuracy: 0.7973
Epoch 2/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4427 - accuracy: 0.7948 - val_loss: 0.4387 - val_accuracy: 0.7927
Epoch 3/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4329 - accuracy: 0.8008 - val_loss: 0.4161 - val_accuracy: 0.8094
Epoch 4/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4257 - accuracy: 0.8049 - val_loss: 0.4072 - val_accuracy: 0.8163
Epoch 5/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4205 - accuracy: 0.8054 - val_loss: 0.4097 - val_accuracy: 0.8144
Epoch 6/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4130 - accuracy: 0.8083 - val_loss: 0.4085 - val_accuracy: 0.8207
Epoch 7/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4089 - accuracy: 0.8120 - val_loss: 0.4065 - val_accuracy: 0.8144
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.5

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 7s 6ms/step - loss: 0.5264 - accuracy: 0.7477 - val_loss: 0.4561 - val_accuracy: 0.7959
Epoch 2/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4832 - accuracy: 0.7812 - val_loss: 0.4500 - val_accuracy: 0.8006
Epoch 3/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4715 - accuracy: 0.7855 - val_loss: 0.4444 - val_accuracy: 0.8004
Epoch 4/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4665 - accuracy: 0.7885 - val_loss: 0.4361 - val_accuracy: 0.8012
Epoch 5/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4627 - accuracy: 0.7915 - val_loss: 0.4414 - val_accuracy: 0.8073
Epoch 6/10
902/902 [==============================] - 6s 6ms/step - loss: 0.4624 - accuracy: 0.7946 - val_loss: 0.4221 - val_accuracy: 0.8108
Epoch 7/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4620 - accuracy: 0.7934 - val_loss: 0.4589 - val_accuracy: 0.8080
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.05

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 8s 6ms/step - loss: 0.4800 - accuracy: 0.7747 - val_loss: 0.4525 - val_accuracy: 0.7976
Epoch 2/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4460 - accuracy: 0.7952 - val_loss: 0.4235 - val_accuracy: 0.8024
Epoch 3/10
902/902 [==============================] - 6s 6ms/step - loss: 0.4352 - accuracy: 0.8001 - val_loss: 0.4207 - val_accuracy: 0.8085
Epoch 4/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4285 - accuracy: 0.8043 - val_loss: 0.4268 - val_accuracy: 0.8051
Epoch 5/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4214 - accuracy: 0.8069 - val_loss: 0.4084 - val_accuracy: 0.8174
Epoch 6/10
902/902 [==============================] - 6s 6ms/step - loss: 0.4202 - accuracy: 0.8088 - val_loss: 0.4062 - val_accuracy: 0.8156
Epoch 7/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4132 - accuracy: 0.8123 - val_loss: 0.4124 - val_accuracy: 0.8090
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.1

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 8s 7ms/step - loss: 0.4855 - accuracy: 0.7752 - val_loss: 0.4403 - val_accuracy: 0.8009
Epoch 2/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4515 - accuracy: 0.7917 - val_loss: 0.4298 - val_accuracy: 0.8020
Epoch 3/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4384 - accuracy: 0.8007 - val_loss: 0.4307 - val_accuracy: 0.8015
Epoch 4/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4315 - accuracy: 0.8016 - val_loss: 0.4396 - val_accuracy: 0.7952
Epoch 5/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4298 - accuracy: 0.8043 - val_loss: 0.4161 - val_accuracy: 0.8142
Epoch 6/10
902/902 [==============================] - 6s 6ms/step - loss: 0.4241 - accuracy: 0.8057 - val_loss: 0.4164 - val_accuracy: 0.8088
Epoch 7/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4175 - accuracy: 0.8078 - val_loss: 0.4128 - val_accuracy: 0.8151
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.1

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 11s 8ms/step - loss: 0.5018 - accuracy: 0.7621 - val_loss: 0.4561 - val_accuracy: 0.7923
Epoch 2/10
902/902 [==============================] - 7s 7ms/step - loss: 0.4618 - accuracy: 0.7894 - val_loss: 0.4456 - val_accuracy: 0.7991
Epoch 3/10
902/902 [==============================] - 7s 8ms/step - loss: 0.4505 - accuracy: 0.7955 - val_loss: 0.4273 - val_accuracy: 0.8002
Epoch 4/10
902/902 [==============================] - 7s 7ms/step - loss: 0.4431 - accuracy: 0.7980 - val_loss: 0.4298 - val_accuracy: 0.8037
Epoch 5/10
902/902 [==============================] - 7s 7ms/step - loss: 0.4454 - accuracy: 0.7955 - val_loss: 0.4484 - val_accuracy: 0.8123
Epoch 6/10
902/902 [==============================] - 6s 7ms/step - loss: 0.4347 - accuracy: 0.8012 - val_loss: 0.4155 - val_accuracy: 0.8105
Epoch 7/10
902/902 [==============================] - 7s 8ms/step - loss: 0.4311 - accuracy: 0.8041 - val_loss: 0.4161 - val_accuracy: 0.8173
Epoch

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 1024
dropout = 0.05

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 8s 6ms/step - loss: 0.4852 - accuracy: 0.7732 - val_loss: 0.4493 - val_accuracy: 0.7907
Epoch 2/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4502 - accuracy: 0.7931 - val_loss: 0.4257 - val_accuracy: 0.8067
Epoch 3/10
902/902 [==============================] - 6s 6ms/step - loss: 0.4360 - accuracy: 0.7989 - val_loss: 0.4247 - val_accuracy: 0.8115
Epoch 4/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4319 - accuracy: 0.8037 - val_loss: 0.4203 - val_accuracy: 0.8095
Epoch 5/10
902/902 [==============================] - 5s 6ms/step - loss: 0.4247 - accuracy: 0.8052 - val_loss: 0.4114 - val_accuracy: 0.8142
Epoch 6/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4198 - accuracy: 0.8083 - val_loss: 0.4109 - val_accuracy: 0.8152
Epoch 7/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4168 - accuracy: 0.8100 - val_loss: 0.4022 - val_accuracy: 0.8167
Epoch 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ffnn_neurons = 512
dropout = 0.05

# Define FFNN model without embedding layer
ffnn_model = Sequential()
ffnn_model.add(Dense(ffnn_neurons, activation='relu', input_shape=(input,)))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(ffnn_neurons, activation='relu'))
ffnn_model.add(Dropout(dropout))
ffnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
ffnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Assuming X_train and y_train1 are defined elsewhere
# Train the model
history = ffnn_model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_val, y_val))


Epoch 1/10
902/902 [==============================] - 7s 5ms/step - loss: 0.4715 - accuracy: 0.7806 - val_loss: 0.4334 - val_accuracy: 0.8083
Epoch 2/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4324 - accuracy: 0.8041 - val_loss: 0.4206 - val_accuracy: 0.8087
Epoch 3/10
902/902 [==============================] - 4s 5ms/step - loss: 0.4187 - accuracy: 0.8074 - val_loss: 0.4068 - val_accuracy: 0.8144
Epoch 4/10
902/902 [==============================] - 5s 5ms/step - loss: 0.4093 - accuracy: 0.8142 - val_loss: 0.3996 - val_accuracy: 0.8217
Epoch 5/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4042 - accuracy: 0.8156 - val_loss: 0.4109 - val_accuracy: 0.8119
Epoch 6/10
902/902 [==============================] - 4s 4ms/step - loss: 0.4009 - accuracy: 0.8153 - val_loss: 0.3934 - val_accuracy: 0.8231
Epoch 7/10
902/902 [==============================] - 5s 6ms/step - loss: 0.3960 - accuracy: 0.8195 - val_loss: 0.3922 - val_accuracy: 0.8234
Epoch 